# Regression Model

### Importing Libraries

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir("drive/MyDrive/DL_Project")

In [3]:
from Dataset import MyDataset
import torch
import torch.nn as nn
import torchvision.transforms as transforms

In [4]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Implementing Model

In [5]:
import torch
import torch.nn as nn
import torchvision.models as models

# Define attention mechanism
class AttentionLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(AttentionLayer, self).__init__()
        self.attention = nn.Sequential(
            nn.Linear(in_features, out_features),
            nn.Sigmoid()
        )

    def forward(self, x):
        attn_weights = self.attention(x)
        return x * attn_weights

# Custom CNN model with ResNet18 and attention layer
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.resnet = models.resnet18(pretrained=True)
        self.attention = AttentionLayer(512, 512)  # Assuming ResNet18 output features are 512
        self.fc = nn.Linear(512, num_classes)  # Single neuron for prediction

    def forward(self, x):
        x = self.resnet.conv1(x)
        x = self.resnet.bn1(x)
        x = self.resnet.relu(x)
        x = self.resnet.maxpool(x)

        x = self.resnet.layer1(x)
        x = self.resnet.layer2(x)
        x = self.resnet.layer3(x)
        x = self.resnet.layer4(x)

        x = self.resnet.avgpool(x)
        x = torch.flatten(x, 1)

        x = self.attention(x)  # Apply attention
        x = self.fc(x)  # Final output

        return x.view(-1,1)

model = CNN(1)
model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 91.6MB/s]


CNN(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_

### Setting Hyperparameters

In [6]:
num_epochs=50
batch_size=2
learning_rate=0.00005

### Defining Loss Function and Optimizer

In [7]:
criterion=nn.MSELoss()
optimizer=torch.optim.AdamW(model.parameters(),lr=learning_rate)

### Training & Validating Model

In [8]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [9]:
train_dataset=MyDataset("Data/train_set.csv",transform=transform)
train_dataloader=torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,shuffle=True)

In [10]:
valid_dataset=MyDataset("Data/validation_set.csv",transform=transform)
valid_dataloader=torch.utils.data.DataLoader(valid_dataset,batch_size=batch_size,shuffle=True)

In [11]:
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for i, (images, target) in enumerate(train_dataloader):
        images = images.reshape(-1,3,4000,3000).to(device)
        target = target.to(device).view(-1,1)

        optimizer.zero_grad()

        output = model(images)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        avg_loss=total_loss/len(train_dataloader)

    # Performing validation
    model.eval()
    with torch.no_grad():
        val_loss = 0.0

        for i, (val_images, val_target) in enumerate(valid_dataloader):
            val_images = val_images.reshape(-1,3,4000,3000).to(device)
            val_target = val_target.to(device).view(-1,1)

            val_output = model(val_images)
            val_loss += criterion(val_output, val_target).item()


            average_val_loss = val_loss / len(valid_dataloader)

    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_loss:.4f}, Validation Loss: {average_val_loss:.4f}')

Epoch [1/50], Training Loss: 1.0690, Validation Loss: 0.4383
Epoch [2/50], Training Loss: 0.6246, Validation Loss: 0.1907
Epoch [3/50], Training Loss: 0.4692, Validation Loss: 0.2344
Epoch [4/50], Training Loss: 0.4599, Validation Loss: 0.2050
Epoch [5/50], Training Loss: 0.3047, Validation Loss: 0.2261
Epoch [6/50], Training Loss: 0.2442, Validation Loss: 0.2452
Epoch [7/50], Training Loss: 0.1970, Validation Loss: 0.3029


KeyboardInterrupt: ignored

### Saving Model Weights

In [ ]:
torch.save(model.state_dict(), 'model_weights.pth')

In [ ]:
model.load_state_dict(torch.load('model_weights.pth'))

### Testing Model

In [12]:
test_dataset=MyDataset("Data/test_set.csv",transform=transform)
test_dataloader=torch.utils.data.DataLoader(test_dataset,batch_size=batch_size,shuffle=True)

In [13]:
correct=[]
predicted=[]
model.eval()
with torch.no_grad():
    test_loss = 0.0

    for i, (test_images, test_target) in enumerate(test_dataloader):
        test_images = test_images.reshape(-1,3,4000,3000).to(device)
        test_target = test_target.to(device).view(-1,1)

        test_output = model(test_images)
        test_loss += criterion(test_output, test_target).item()

        correct.append(test_target)
        predicted.append(test_output)

        average_test_loss = test_loss / len(test_dataloader)

    print(f'Average Loss: {average_test_loss}')

Average Loss: 0.5370181113378041


In [15]:
for x,y in zip(correct,predicted):
  for i,j in zip(x,y):
    print(i.item(),j.item())

0.28999999165534973 -0.2531120479106903
1.0700000524520874 1.207817554473877
0.574999988079071 0.3017595708370209
4.050000190734863 2.532141923904419
2.125 1.446120262145996
0.8349999785423279 0.13314920663833618
3.4549999237060547 2.9923553466796875
2.125 1.313530445098877
1.940000057220459 1.665525197982788
2.2950000762939453 1.37776780128479
1.774999976158142 1.953223466873169
2.0199999809265137 1.7800241708755493
2.5350000858306885 1.7730950117111206
1.8350000381469727 1.2665289640426636
3.265000104904175 2.1005730628967285
2.450000047683716 1.701753854751587
3.434999942779541 2.6319241523742676
3.180000066757202 2.950305700302124
1.8350000381469727 1.0309303998947144
2.1500000953674316 1.4740568399429321
1.5549999475479126 0.7733040452003479
0.6650000214576721 0.6836261749267578
2.7100000381469727 2.4391837120056152
2.0399999618530273 1.3872705698013306
2.0199999809265137 1.0657202005386353
1.5800000429153442 0.975288987159729
0.46000000834465027 0.0675409734249115
1.7549999952316